# snnTorch - Tutorial 3
### By Jason K. Eshraghian and Gregor Lenz

# Neuromorphic datasets
Now we're going to look at how we can use datasets that were recorded with a neuromorphic camera. For that we make use of [Tonic](https://github.com/neuromorphs/tonic), which works much like PyTorch vision. We can simply install the package from pypi.

In [ ]:
!pip install tonic

Let's start by loading the neuromorphic version of the MNIST dataset which is called [N-MNIST](https://tonic.readthedocs.io/en/latest/datasets.html#n-mnist). We can have a look at some raw events to get a feeling what we're working with. The raw output is an NxD array of events. Every row is one event and the columns are for x and y coordinates, timestamp and polarity.

In [ ]:
import tonic

dataset = tonic.datasets.NMNIST(save_to='./data', train=False, download=True,)
events, target = dataset[0]
print(events)

If we were to accumulate those events over time and plot the bins as images, it looks like this:

In [ ]:
tonic.utils.plot_event_grid(events, dataset.ordering)

Our neural network doesn't just take as an input an array of raw events. We want to convert the raw data to a representation that is suitable, such as a tensor. We can choose a set of transforms to apply to our data before feeding it to our network. The neuromorphic camera sensor has a temporal resolution of microseconds, which when converted into a dense representation ends up in a very large tensor. That is why we specify a [Downsample](https://tonic.readthedocs.io/en/latest/transformations.html#downsample-timestamps-and-or-spatial-coordinates) transform to reduce temporal resolution to milliseconds, which is sufficient for our task. Then we convert the raw array of events into a sparse tensor from which the dense version can easily be generated.

In [ ]:
import tonic.transforms as transforms

transform = transforms.Compose([transforms.Downsample(time_factor=1e-3),
                                transforms.ToSparseTensor(merge_polarities=False),]) # set this to True if you want a single channel

download = True
trainset = tonic.datasets.NMNIST(save_to='./data', download=download, transform=transform, train=True)
testset = tonic.datasets.NMNIST(save_to='./data', download=download, transform=transform, train=False)

Now we can start loading data much like we would load from any other Pytorch dataloader. Because SNNTorch uses time as its first dimension, we have to permute the tensor dimensions from (Batch, Time, Channels, Width, Height) to (Time, Batch, Channels, Width, Height). 

In [ ]:
from torch.utils.data import DataLoader
trainloader = DataLoader(trainset, shuffle=True)

event_tensor, target = next(iter(trainloader))

In [ ]:
event_tensor

In [ ]:
print(event_tensor.to_dense().permute([1,0,2,3,4]).shape)
# feed this to the network
event_tensor.to_dense().permute([1,0,2,3,4])